In [52]:
%%bash 

# Removing all tables that were previously downloaded:
rm *tsv

# Setting date tag:
today=$(date '+%Y-%m-%d')

# Fetching associations:
wget -q https://www.ebi.ac.uk/gwas/api/search/downloads/alternative \
    -O gwas_associations_${today}.tsv
    
# Fetching studies:
wget -q  https://www.ebi.ac.uk/gwas/api/search/downloads/unpublished_studies    \
    -O gwas_unpublished_studies_${today}.tsv

wget -q https://www.ebi.ac.uk/gwas/api/search/downloads/studies_new \
    -O gwas_studies_new_${today}.tsv

# Old style study table, not needed anymore
# wget -q https://www.ebi.ac.uk/gwas/api/search/downloads/studies_alternative \
#     -O gwas_studies_${today}.tsv

# Downloading ancestries:
wget -q https://www.ebi.ac.uk/gwas/api/search/downloads/ancestries_new \
    -O gwas_ancestries_new_${today}.tsv

wget -q  https://www.ebi.ac.uk/gwas/api/search/downloads/unpublished_ancestries   \
    -O gwas_unpublished_ancestries_${today}.tsv

# Also obsoleted dataset
# wget -q  https://www.ebi.ac.uk/gwas/api/search/downloads/ancestry    \
#     -O gwas_ancestries_${today}.tsv


ls -lah *tsv

-rw-r--r--  1 dsuveges  384566875    11M 27 May 10:49 gwas_ancestries_new_2022-05-27.tsv
-rw-r--r--  1 dsuveges  384566875   229M 27 May 10:49 gwas_associations_2022-05-27.tsv
-rw-r--r--  1 dsuveges  384566875    19M 27 May 10:49 gwas_studies_new_2022-05-27.tsv
-rw-r--r--  1 dsuveges  384566875   741K 27 May 10:49 gwas_unpublished_ancestries_2022-05-27.tsv
-rw-r--r--  1 dsuveges  384566875   456K 27 May 10:49 gwas_unpublished_studies_2022-05-27.tsv


In [45]:
import pyspark.sql
import pyspark.sql.types as t
import pyspark.sql.functions as f

global spark

spark = (pyspark.sql.SparkSession
       .builder
       .master("local[*]")
       .getOrCreate()
)
        
# Reading GWAS Catalog data into spark dataframe
studies = spark.read.csv('gwas_ancestries_new_2022-05-24.tsv', sep='\t', header=True).persist()
ustudies = spark.read.csv('gwas_unpublished_ancestries_2022-05-24.tsv', sep='\t', header=True).persist()

In [48]:
all_studies = (
    studies.union(ustudies)
    .persist()
)

all_studies.count()



63037

In [49]:
studies.count()

57229

In [50]:
ustudies.count()

5808

In [43]:
ustudies.columns

['STUDY ACCESSION',
 'PUBMED ID',
 'FIRST AUTHOR',
 'DATE',
 'INITIAL SAMPLE DESCRIPTION',
 'REPLICATION SAMPLE DESCRIPTION',
 'STAGE',
 'NUMBER OF INDIVIDUALS',
 'BROAD ANCESTRAL CATEGORY',
 'COUNTRY OF ORIGIN',
 'COUNTRY OF RECRUITMENT',
 'ADDITIONAL ANCESTRY DESCRIPTION',
 'ANCESTRY DESCRIPTOR',
 'FOUNDER/GENETICALLY ISOLATED POPULATION',
 'NUMBER OF CASES',
 'NUMBER OF CONTROLS',
 'SAMPLE DESCRIPTION',
 'COHORT(S)',
 'COHORT-SPECIFIC REFERENCE']